# Find the Top Ten Words of Recall words for each wv
## "Third" bc this is the third notebook that does this, but in this one, we regress out different things (29):
### L_corr = corr(L, recall - story)
### S_corr = corr(S, recall - story)
### story_corr = corr(story, recall - L - S - other stories with shared schema)
### ) all other possible regressors besides the regressor used in the correlation are used in the initial regression

## 2/18/20
### For each story there will be 3 'Top Ten Words', One for each type of regressor
#### 1) Concatenate unique words across all participants
#### 2) For each regressor, regress out of the words the other vector(s). 
#### 3) Correlate the residual wvs with the target regressor. 
#### 4) Index the 10 highest values, and find the 10 highest words
#### 5) Put in dictionary



In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
import os
import glob
import math
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import copy
from random import randrange
from sklearn.metrics import jaccard_score
import random
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression
from scipy import stats


from sklearn.decomposition import PCA #for cluster analysis
from gensim.models import KeyedVectors #for word embeddings
import matplotlib.pyplot as plt #for plotting
import os #for importing
import pickle #for loading transcripts
from scipy.stats import pearsonr 

# from _DRAFT_20200604_functions import * #includes constants and score function
from tqdm import tqdm_notebook #for progress bar

%autosave 5

Autosaving every 5 seconds


# 0. Import recalls, and uncentered story and template vectors and sums 

In [2]:
# import recalls and sums
recalls = pickle.load( open( 'fr_recalls', "rb" ) )
sums = pickle.load( open( "fr_sums", "rb" ) )

#import non-centered story, template
templates = pickle.load( open( 'template_vectors', "rb" ) )
stories = pickle.load( open( 'actual_story_vectors', "rb" ) )

In [3]:
top_ten = {}

In [4]:
wikipath = 'rolando/wiki-news-300d-1M.vec'
wv_model = KeyedVectors.load_word2vec_format(wikipath)

In [5]:
wv_dim = 300

# FastText preprocessing, based on bittlingmayer/ft_wiki_preproc.py
# Remove special characters, put spaces between all tokens
SUB = ["s/’/'/g", "s/′/'/g", "s/''/ /g", "s/'/ ' /g", 's/“/"/g', 's/”/"/g', 's/"/ /g', "s/\\./ \\. /g", "s/<br \\/>/ /g", "s/, / , /g", "s/(/ ( /g", "s/)/ ) /g", "s/\\!/ \\! /g", "s/\\?/ \\? /g", "s/\\;/ /g", "s/\\:/ /g", "s/-/ - /g", "s/=/ /g", "s/=/ /g", "s/*/ /g", "s/|/ /g", "s/«/ /g", 
       "s/…/ /g", "s/‘/ /g", "s/í/ /g", "s/ñ/ /g", "s/\x84/ /g", "s/î/ /g", "s/ó/ /g", "s/\x83/ /g", "s/ï/ /g", "s/õ/ /g",
       "s/ò/ /g", "s/,/ /g", "s/ô/ /g", "s/\x92/ /g", "s/é/ /g", "s/\x8e/ /g", "s/â\x80¦/ /g", "s/\x91/ /g", "s/\x93/ /g",
       "s/\x94/ /g", "s/ã®/ /g", "s/ã¨/ /g", "s/ã©/ /g",
       "s/\â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x9d/ /g", "s/â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x98/ /g",
       "s/â/ /g"]

def __normalize_text(s):
    for sg in SUB:
        rep = sg.replace('\\','').split('/')
        s = s.replace(rep[1], rep[2])
    s = s.replace('/',' ')
    return s

def __spaces(s):
    return ' '.join(s.split())

def __digits(s):
    return ''.join(filter(lambda c: not c.isdigit(), s))

# def preproc(s):
#     return __punctuation(__spaces(__digits(__normalize_text(s.lower()))))

def preproc(s):
    return (__spaces(__digits(__normalize_text(s.lower()))))

def __punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))

def word2vecSent(sentence, model = 'fasttext'):
    wv_dim = 300 #for glove and fasttext
    
    if model == 'glove':
        wvmodel = glove_model
    elif model == 'fasttext':
        wvmodel = wv_model
        
    words = preproc(sentence).split(' ')
    wv = np.zeros((len(words), wv_dim))
    for i in range(len(words)):
        if words[i] in wvmodel.vocab:
            wv[i,:] = wvmodel.word_vec(words[i])
    
    return words, wv

# I. 3 sets of top ten 
## A. Find unique words in recalls of each story
### 1. Concatenate words and word vectors in parallel across all participants

In [6]:
all_words = {}
all_wvs = {}

#iterate through all recalls in a story and concatente words and wvs of all stories
for key in recalls:
    words = np.zeros((0,1))
    wvs = np.zeros((0,300))
    for i in range(0,3):
        for j in range(0,len(recalls[key][i])):
            p_words, p_wvs = word2vecSent(recalls[key][i][j][0])
            # reshape p_words
            p_words = np.array(p_words)
            p_words = p_words.reshape(p_words.shape[0],-1)
            # stack p_words and p_wvs
            words = np.vstack((words, p_words))
            wvs = np.vstack((wvs, p_wvs))
    all_words[key] = words
    all_wvs[key] = wvs

### 2. Centering all words + story wvs + template wvs

In [7]:
#Concatenate all words from recall

concat = np.zeros((0,300))

for key in all_wvs:
    for i in range(0, len(all_wvs[key])):
        concat = np.vstack((concat, all_wvs[key][i]))
    


In [8]:
print(concat.shape)
for key in stories:
    concat = np.vstack((concat, stories[key]))

(37545, 300)


In [9]:
print(concat.shape)
for key in templates:
    concat = np.vstack((concat, templates[key]))
print(concat.shape)

(37561, 300)
(37569, 300)


In [10]:
centering_vec = np.mean(concat, axis = 0)

# template vectors

for key in templates:
    templates[key] = templates[key] - centering_vec
    
# recall vectors

for key in recalls:
    for i in range(0, len(all_wvs[key])):
        all_wvs[key][i] = all_wvs[key][i] - centering_vec
            
#story vectors
#make new dict with int key instead of string
int_stories = {}
for key in stories:
    int_stories[int(key)] = stories[key] - centering_vec

### 3. Find unique words and wvs

In [11]:
au_words = {}
au_wvs = {}

for key in all_words:
    # find unique wvs and their indices:
    unique_wvs, index = np.unique(all_wvs[key], axis=0, return_index=True)
    unique_words =  all_words[key][index]
    # put in dicts
    au_wvs[key] = unique_wvs
    au_words[key] = unique_words

## B. *Story* Top Ten
### A. Regress out of all the unique_wvs the loc and soc template wvs
#### i. Concatenating the schema part of the matrix used as input to regression

In [41]:
schema_order = [1, 2, 3, 4, 10, 20, 30, 40]

this_input2 = np.zeros((300,0))

for i in schema_order:
    this_input2 = np.concatenate((this_input2, templates[i].reshape(templates[i].shape[0],-1)), axis = 1)

### 1. 8 regressors total: L + S templates + 6 schema sharing stories

In [35]:
story_order = [11,12,13,14,21,22,23,24,31,32,33,34,41,42,43,44]
resid_wvs = {}

for key in au_wvs:
    these_residuals = np.zeros((0, 300))
    # add on all other stories to the regression input
    loc = key%10
    soc = round(key/10)*10
    # location template
    l_temp = templates[loc].reshape(templates[loc].shape[0],-1)
    # social template
    s_temp = templates[soc].reshape(templates[soc].shape[0],-1)
    inputs = np.concatenate((l_temp, s_temp), axis = 1)
    # List the set of stories that share a schema with the stories
    shared_schema = []
    for story in story_order:
        this_loc = story%10
        this_soc = round(story/10)*10
        if story == key:
            pass
        elif this_loc == loc:
            shared_schema.append(story)
        elif this_soc == soc:
            shared_schema.append(story)
        #else:
        #    shared_schema.append(story)
    # Iterate through the list
    for story in shared_schema:
        inputs = np.concatenate((inputs, int_stories[story].reshape(int_stories[story].shape[0],-1)), axis = 1)
    print(inputs.shape)
    for i in range(0,len(au_wvs[key])):
        output = np.copy(au_wvs[key][i])
        #output = output.reshape(output.shape[0],-1)
        model = LinearRegression().fit(inputs, output)
        ## Is subtracting the intercept correct?? 
        new_vec = output - np.dot(model.coef_, inputs.T)- model.intercept_
        these_residuals = np.vstack((these_residuals, new_vec))
    resid_wvs[key] = these_residuals

(300, 17)
(300, 17)
(300, 17)
(300, 17)
(300, 17)
(300, 17)
(300, 17)
(300, 17)
(300, 17)
(300, 17)
(300, 17)
(300, 17)
(300, 17)
(300, 17)
(300, 17)
(300, 17)


### 2. 8 regressors total: only the templates

In [42]:
story_order = [11,12,13,14,21,22,23,24,31,32,33,34,41,42,43,44]
resid_wvs = {}

for key in au_wvs:
    these_residuals = np.zeros((0, 300))
    #inputs
    inputs = np.copy(this_input2)
    print(inputs.shape)
    for i in range(0,len(au_wvs[key])):
        output = np.copy(au_wvs[key][i])
        #output = output.reshape(output.shape[0],-1)
        model = LinearRegression().fit(inputs, output)
        ## Is subtracting the intercept correct?? 
        new_vec = output - np.dot(model.coef_, inputs.T)- model.intercept_
        these_residuals = np.vstack((these_residuals, new_vec))
    resid_wvs[key] = these_residuals

(300, 8)
(300, 8)
(300, 8)
(300, 8)
(300, 8)
(300, 8)
(300, 8)
(300, 8)
(300, 8)
(300, 8)
(300, 8)
(300, 8)
(300, 8)
(300, 8)
(300, 8)
(300, 8)


### 3. 23 regressors: all possible regressors

In [45]:
    
story_order = [11,12,13,14,21,22,23,24,31,32,33,34,41,42,43,44]
resid_wvs = {}

for key in au_wvs:
    these_residuals = np.zeros((0, 300))
    #inputs
    inputs = np.copy(this_input2)
    for story in story_order:
        if story == key:
            print('skip')
        else:
            inputs = np.concatenate((inputs, stories[str(story)].reshape(stories[str(story)].shape[0],-1)), axis = 1)
    print(inputs.shape)
    for i in range(0,len(au_wvs[key])):
        output = np.copy(au_wvs[key][i])
        #output = output.reshape(output.shape[0],-1)
        model = LinearRegression().fit(inputs, output)
        ## Is subtracting the intercept correct?? 
        new_vec = output - np.dot(model.coef_, inputs.T)- model.intercept_
        these_residuals = np.vstack((these_residuals, new_vec))
    resid_wvs[key] = these_residuals

skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)
skip
(300, 23)


### 4. 14 Regressors: The 8 schemas + 6 related stories

In [51]:
story_order = [11,12,13,14,21,22,23,24,31,32,33,34,41,42,43,44]
resid_wvs = {}

for key in au_wvs:
    these_residuals = np.zeros((0, 300))
    #inputs
    inputs = np.copy(this_input2)
    print(inputs.shape)
    loc = key%10
    soc = round(key/10)*10
    # List the set of stories that share a schema with the stories
    shared_schema = []
    for story in story_order:
        this_loc = story%10
        this_soc = round(story/10)*10
        if story == key:
            pass
        elif this_loc == loc:
            shared_schema.append(story)
        elif this_soc == soc:
            shared_schema.append(story)
    # Iterate through the list
    print(inputs.shape)
    for story in shared_schema:
        inputs = np.concatenate((inputs, int_stories[story].reshape(int_stories[story].shape[0],-1)), axis = 1)
    print(inputs.shape)
    for i in range(0,len(au_wvs[key])):
        output = np.copy(au_wvs[key][i])
        #output = output.reshape(output.shape[0],-1)
        model = LinearRegression().fit(inputs, output)
        ## Is subtracting the intercept correct?? 
        new_vec = output - np.dot(model.coef_, inputs.T)- model.intercept_
        these_residuals = np.vstack((these_residuals, new_vec))
    resid_wvs[key] = these_residuals

(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)
(300, 8)
(300, 8)
(300, 14)


### 5. 15 regressors: only all the stories


In [58]:
    
story_order = [11,12,13,14,21,22,23,24,31,32,33,34,41,42,43,44]
resid_wvs = {}

for key in au_wvs:
    these_residuals = np.zeros((0, 300))
    #inputs
    inputs = np.zeros((300,0))
    for story in story_order:
        if story == key:
            print('skip')
        else:
            inputs = np.concatenate((inputs, stories[str(story)].reshape(stories[str(story)].shape[0],-1)), axis = 1)
    print(inputs.shape)
    for i in range(0,len(au_wvs[key])):
        output = np.copy(au_wvs[key][i])
        #output = output.reshape(output.shape[0],-1)
        model = LinearRegression().fit(inputs, output)
        ## Is subtracting the intercept correct?? 
        new_vec = output - np.dot(model.coef_, inputs.T)- model.intercept_
        these_residuals = np.vstack((these_residuals, new_vec))
    resid_wvs[key] = these_residuals

skip
(300, 15)
skip
(300, 15)
skip
(300, 15)
skip
(300, 15)
skip
(300, 15)
skip
(300, 15)
skip
(300, 15)
skip
(300, 15)
skip
(300, 15)
skip
(300, 15)
skip
(300, 15)
skip
(300, 15)
skip
(300, 15)
skip
(300, 15)
skip
(300, 15)
skip
(300, 15)


### 6. 9 Regressors: The stories that don't share a schema

In [62]:
story_order = [11,12,13,14,21,22,23,24,31,32,33,34,41,42,43,44]
resid_wvs = {}

for key in au_wvs:
    these_residuals = np.zeros((0, 300))
    #inputs
    inputs = np.zeros((300,0))
    loc = key%10
    soc = round(key/10)*10
    # List the set of stories that share a schema with the stories
    shared_schema = []
    for story in story_order:
        this_loc = story%10
        this_soc = round(story/10)*10
        if story == key:
            pass
        elif this_loc == loc:
            pass
        elif this_soc == soc:
            pass
        else:
            shared_schema.append(story)
    # Iterate through the list
    for story in shared_schema:
        inputs = np.concatenate((inputs, int_stories[story].reshape(int_stories[story].shape[0],-1)), axis = 1)
    print(inputs.shape)
    for i in range(0,len(au_wvs[key])):
        output = np.copy(au_wvs[key][i])
        #output = output.reshape(output.shape[0],-1)
        model = LinearRegression().fit(inputs, output)
        ## Is subtracting the intercept correct?? 
        new_vec = output - np.dot(model.coef_, inputs.T)- model.intercept_
        these_residuals = np.vstack((these_residuals, new_vec))
    resid_wvs[key] = these_residuals

(300, 9)
(300, 9)
(300, 9)
(300, 9)
(300, 9)
(300, 9)
(300, 9)
(300, 9)
(300, 9)
(300, 9)
(300, 9)
(300, 9)
(300, 9)
(300, 9)
(300, 9)
(300, 9)


### 7. 17 Regressors: 15 stories + Loc and Soc Templates

In [67]:
story_order = [11,12,13,14,21,22,23,24,31,32,33,34,41,42,43,44]
resid_wvs = {}

for key in au_wvs:
    these_residuals = np.zeros((0, 300))
    # add on all other stories to the regression input
    loc = key%10
    soc = round(key/10)*10
    # location template
    l_temp = templates[loc].reshape(templates[loc].shape[0],-1)
    # social template
    s_temp = templates[soc].reshape(templates[soc].shape[0],-1)
    inputs = np.concatenate((l_temp, s_temp), axis = 1)
    # List the set of stories that share a schema with the stories
    for story in story_order:
        if story == key:
            print('skip')
        else:
            inputs = np.concatenate((inputs, stories[str(story)].reshape(stories[str(story)].shape[0],-1)), axis = 1)
    print(inputs.shape)
    for i in range(0,len(au_wvs[key])):
        output = np.copy(au_wvs[key][i])
        #output = output.reshape(output.shape[0],-1)
        model = LinearRegression().fit(inputs, output)
        ## Is subtracting the intercept correct?? 
        new_vec = output - np.dot(model.coef_, inputs.T)- model.intercept_
        these_residuals = np.vstack((these_residuals, new_vec))
    resid_wvs[key] = these_residuals

skip
(300, 17)
skip
(300, 17)
skip
(300, 17)
skip
(300, 17)
skip
(300, 17)
skip
(300, 17)
skip
(300, 17)
skip
(300, 17)
skip
(300, 17)
skip
(300, 17)
skip
(300, 17)
skip
(300, 17)
skip
(300, 17)
skip
(300, 17)
skip
(300, 17)
skip
(300, 17)


### C. Correlate each wv in the recall with the story wv

In [68]:
all_correls = {}

for key in resid_wvs:
    corr = np.zeros((0,1))
    this_story = stories[str(key)].reshape(stories[str(key)].shape[0], -1)
    for i in range(0, len(resid_wvs[key])):
        this_wv = resid_wvs[key][i].reshape(resid_wvs[key][i].shape[0], -1)
        this_correl = np.corrcoef(this_story.T, this_wv.T)
        # add to corr
        corr = np.vstack((corr, this_correl[0,1]))
    all_correls[key] = corr

### D. Top Ten values' indices and words

In [69]:
for key in all_correls:
    print(key)
    this_corr = all_correls[key]
    this_corr = np.ravel(this_corr)
    story_index = this_corr.argsort()[-10:][::-1]
    #print(story_index)
    # index the top words
    top_words = np.zeros((0,1))
    for i in range(0, 10):
        top_words = np.vstack((top_words, au_words[key][story_index[i]]))
    top_words = top_words.ravel()
    print(top_words)
    top_ten[key] = {'story': top_words}

11
['thomas' 'got' 't' "'" 'sat' 'hadn' 'ate' 'started' 'burguer' 'had']
33
['ms' 'cheeses' 'stores' 'cheese' 'product' 'products' 'morrison' '.'
 'crackers' 'marketed']
44
['during' 'saw' 'egyptology' 'found' 'wrote' 'had' 'mummification' 'took'
 'heard' 'looked']
23
['alex' 'chloe' 'loves' 'excitement' 'likes' 'chris' 'everyone' 'shoppers'
 'he' 'day']
12
['calvin' 'they' 'ultimately' 'not' 'delayed' 'would' 'flight' 'was' 't'
 'conveyor']
21
['senna' 'oysters' 'champagne' 'dishes' 'delicious' 'caramelized'
 'scallops' 'tears' 'garlic' 'delight']
13
['tonight' 'think' 'going' 'well' 'however' 'agreeing' 'nodded' 'said'
 'not' 'anyway']
42
['she' 'gave' 'anna' 'had' 'her' 'was' 'of' 'might' 'could' 'rate']
43
['carina' 'simon' 'ribs' 'meat' 'cute' 'chickens' 'blushed' 'hens'
 'unlucky' 'met']
32
['jeff' 'william' 'wine' 'steep' 'pass' 'vineyard' 'per' 'd' 'samples'
 'vineyards']
34
['alma' 'fund' 'funding' 'funds' 'investors' 'project' 'investor'
 'capital' 'investment' 'finance']
22


## B. *Location* Top Ten
### 1. Regress out of all the unique_wvs the story and soc template wvs
#### a. Concatenating the story part of the matrix used as input to regression

In [21]:
this_input = np.zeros((300,0))

for story in story_order:
    this_input = np.concatenate((this_input, stories[str(story)].reshape(stories[str(story)].shape[0],-1)), axis = 1)

print(this_input.shape)

(300, 16)


In [24]:
resid_wvs = {}

for key in au_wvs:
    these_residuals = np.zeros((0, 300))
    loc = key%10
    # Make the inputs of the regression
    new_input = int_stories[key].reshape(int_stories[key].shape[0],-1)        
    print(new_input.shape)
    for i in range(0,len(au_wvs[key])):
        output = np.copy(au_wvs[key][i])
        #output = output.reshape(output.shape[0],-1)
        model = LinearRegression().fit(new_input, output)
        ## Is subtracting the intercept correct?? 
        new_vec = output - np.dot(model.coef_, new_input.T)- model.intercept_
        these_residuals = np.vstack((these_residuals, new_vec))
    resid_wvs[key] = these_residuals

(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)


### 2. Correlate each wv in the recall with the story wv

In [25]:
all_correls = {}

for key in resid_wvs:
    corr = np.zeros((0,1))
    loc = key%10
    this_loc = templates[loc].reshape(templates[loc].shape[0], -1)
    for i in range(0, len(resid_wvs[key])):
        this_wv = resid_wvs[key][i].reshape(resid_wvs[key][i].shape[0], -1)
        this_correl = np.corrcoef(this_loc.T, this_wv.T)
        # add to corr
        corr = np.vstack((corr, this_correl[0,1]))
    all_correls[key] = corr


### 3. Top Ten values' indices and words

In [26]:
for key in all_correls:
    print(key)
    this_corr = all_correls[key]
    this_corr = np.ravel(this_corr)
    story_index = this_corr.argsort()[-10:][::-1]
    #print(story_index)
    # index the top words
    top_words = np.zeros((0,1))
    for i in range(0, 10):
        top_words = np.vstack((top_words, au_words[key][story_index[i]]))
    top_words = top_words.ravel()
    print(top_words)
    top_ten[key]['loc'] =  top_words

11
['restaurant' 'food' 'menus' 'menu' 'diner' 'restuarant' 'waiter' 'foods'
 'meal' 'dessert']
33
['grocery' 'store' 'groceries' 'stores' 'supermarket' 'shop' 'checkout'
 'checkouts' 'cashier' 'purchase']
44
['lecture' 'class' 'lectures' 'lecturing' 'assignment' 'hall' 'classroom'
 'lesson' 'of' 'lecturer']
23
['grocery' 'store' 'groceries' 'supermarket' 'checkout' 'shop' 'shopping'
 'checkouts' 'grocer' 'shoppers']
12
['airport' 'airports' 'plane' 'flight' 'boarding' 'gate' 'gates'
 'security' 'departure' 'arriving']
21
['restaurant' 'food' 'menus' 'menu' 'chef' 'meal' 'dishes' 'waiter'
 'ordering' 'eating']
13
['grocery' 'store' 'groceries' 'checkout' 'shopping' 'cashier' 'items'
 'cart' 'shelves' 'food']
42
['airport' 'plane' 'flight' 'boarding' 'gate' 'airplane' 'security'
 'departure' 'planes' 'destination']
43
['grocery' 'store' 'groceries' 'supermarket' 'checkout' 'shop' 'shopping'
 'warehouse' 'cashier' 'items']
32
['airport' 'plane' 'flight' 'gate' 'flights' 'airplane' 'gates

## C. *Social* Top Ten
### 1. Regress out of all the unique_wvs the loc and soc template wvs

In [29]:
resid_wvs = {}

for key in au_wvs:
    these_residuals = np.zeros((0, 300))
    soc = round(key/10)*10
    # Make the inputs of the regression
    new_input = int_stories[key].reshape(int_stories[key].shape[0],-1)
    print(new_input.shape)
    for i in range(0,len(au_wvs[key])):
        output = np.copy(au_wvs[key][i])
        #output = output.reshape(output.shape[0],-1)
        model = LinearRegression().fit(new_input, output)
        ## Is subtracting the intercept correct?? 
        new_vec = output - np.dot(model.coef_, new_input.T)- model.intercept_
        these_residuals = np.vstack((these_residuals, new_vec))
    resid_wvs[key] = these_residuals

(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)
(300, 1)


### 2. Correlate each wv in the recall with the social wv

In [30]:
all_correls = {}

for key in resid_wvs:
    corr = np.zeros((0,1))
    soc = round(key/10)*10
    this_soc = templates[soc].reshape(templates[soc].shape[0], -1)
    for i in range(0, len(resid_wvs[key])):
        this_wv = resid_wvs[key][i].reshape(resid_wvs[key][i].shape[0], -1)
        this_correl = np.corrcoef(this_soc.T, this_wv.T)
        # add to corr
        corr = np.vstack((corr, this_correl[0,1]))
    all_correls[key] = corr

### D. Top Ten values' indices and words

In [31]:
for key in all_correls:
    this_corr = all_correls[key]
    this_corr = np.ravel(this_corr)
    story_index = this_corr.argsort()[-10:][::-1]
    print(story_index)
    # index the top words
    top_words = np.zeros((0,1))
    for i in range(0, 10):
        top_words = np.vstack((top_words, au_words[key][story_index[i]]))
    top_words = top_words.ravel()
    print(top_words)
    top_ten[key]['soc'] = top_words

[471 162 233 211 109 422 297 445 462 128]
['breakup' 'breaking' 'for' 'break' 'what' 'reason' 'considering' 'the'
 'relationship' 'up']
[256 144 310 442 273 108 471 457 416  69]
['deal' 'business' 'partnership' 'investment' 'agreement' 'success'
 'competitor' 'deals' 'the' 'strategy']
[376 219 248  94 258 220 375 185 280 193]
['the' 'of' 'first' 'what' 'two' 'next' 'time' 'other' 'when' 'for']
[395 364 381 361 320 489 393 536 462 497]
['ring' 'diamond' 'proposal' 'proposing' 'proposed' 'diamonds' 'planning'
 'marriage' 'the' 'engagement']
[401 106 187 452 172 150 372 196 392  70]
['breakup' 'breaking' 'split' 'departure' 'for' 'break' 'the' 'of'
 'relationship' 'what']
[474 362 339 355 294 360 487 447 452 423]
['wedding' 'ring' 'diamond' 'proposal' 'proposed' 'planning' 'marriage'
 'diamonds' 'engagement' 'the']
[487 152 240 106 215 455 266 478 125 222]
['breakup' 'breaking' 'for' 'what' 'break' 'the' 'of' 'relationship' 'up'
 'keeping']
[385 224 302 268 301 384 226 187 290 200]
['the'

# II. Presenting Top Ten Words
## A. Export

In [32]:
pickle.dump( top_ten, open( 'top_ten3', "wb" ) )

## B. Print

In [33]:
story_order = [11,12,13,14,21,22,23,24,31,32,33,34,41,42,43,44]

for story in story_order:
    print('\n\n',story)
    for keys in top_ten[story]:
        print(keys)
        print(top_ten[story][keys])



 11
story
['could' 'looked' 'might' 'hadn' 'don' 'wasn' 'hear' 'thanks' 'seeing'
 'can']
loc
['restaurant' 'food' 'menus' 'menu' 'diner' 'restuarant' 'waiter' 'foods'
 'meal' 'dessert']
soc
['breakup' 'breaking' 'for' 'break' 'what' 'reason' 'considering' 'the'
 'relationship' 'up']


 12
story
['slightly' 'couldn' 'could' 'don' 'onto' 'ago' 'looked' 'might' 'make'
 'can']
loc
['airport' 'airports' 'plane' 'flight' 'boarding' 'gate' 'gates'
 'security' 'departure' 'arriving']
soc
['breakup' 'breaking' 'split' 'departure' 'for' 'break' 'the' 'of'
 'relationship' 'what']


 13
story
['wouldn' 'might' 'could' 'didn' 'doesn' 'wasn' 'nodded' 'don' 'can'
 'looked']
loc
['grocery' 'store' 'groceries' 'checkout' 'shopping' 'cashier' 'items'
 'cart' 'shelves' 'food']
soc
['breakup' 'breaking' 'for' 'what' 'break' 'the' 'of' 'relationship' 'up'
 'keeping']


 14
story
['couldn' 'could' 'don' 'can' 'hear' 'make' 'sorry' 'doesn' 'always'
 'shouldn']
loc
['lecture' 'class' 'lectures' 'teaching' '

In [34]:
top_ten

{11: {'story': array(['could', 'looked', 'might', 'hadn', 'don', 'wasn', 'hear',
         'thanks', 'seeing', 'can'], dtype='<U32'),
  'loc': array(['restaurant', 'food', 'menus', 'menu', 'diner', 'restuarant',
         'waiter', 'foods', 'meal', 'dessert'], dtype='<U32'),
  'soc': array(['breakup', 'breaking', 'for', 'break', 'what', 'reason',
         'considering', 'the', 'relationship', 'up'], dtype='<U32')},
 33: {'story': array(['doesn', 'could', 'wasn', 'isn', 'can', 'make', 'aren', 'would',
         'extremely', 'should'], dtype='<U32'),
  'loc': array(['grocery', 'store', 'groceries', 'stores', 'supermarket', 'shop',
         'checkout', 'checkouts', 'cashier', 'purchase'], dtype='<U32'),
  'soc': array(['deal', 'business', 'partnership', 'investment', 'agreement',
         'success', 'competitor', 'deals', 'the', 'strategy'], dtype='<U32')},
 44: {'story': array(['couldn', 'could', 'doesn', 'don', 'wasn', 'can', 'nodded',
         'knows', 'hasn', 'feels'], dtype='<U32'),
  '